---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [249]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    
    return # Your answer here

In [250]:
#importing train data
df = pd.read_csv('train.csv', encoding = 'ISO-8859-1')
addresses_df = pd.read_csv('addresses.csv')
latlons = pd.read_csv('latlons.csv')

# importing test data
test_df = pd.read_csv('test.csv')

C:\Users\Admin\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [251]:
Y = df.compliance

In [252]:
X = df.drop(columns=['compliance'])

In [134]:
X_train.columns

Index(['ticket_id', 'agency_name', 'inspector_name', 'violator_name',
       'violation_street_number', 'violation_street_name',
       'violation_zip_code', 'mailing_address_str_number',
       'mailing_address_str_name', 'city', 'state', 'zip_code',
       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date',
       'violation_code', 'violation_description', 'disposition', 'fine_amount',
       'admin_fee', 'state_fee', 'late_fee', 'discount_amount',
       'clean_up_cost', 'judgment_amount', 'grafitti_status'],
      dtype='object')

In [119]:
print(addresses_df.shape)
addresses_df.head()

(311307, 2)


,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [121]:
print(latlons.shape)
latlons.head()

(121769, 3)


,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [253]:
addresses_and_latlons = addresses_df.merge(latlons, how='outer', on='address')

In [254]:
print(addresses_and_latlons.shape)
addresses_and_latlons.head()

(311307, 4)


,ticket_id,address,lat,lon
0,22056,"2900 tyler, Detroit MI",42.390729,-83.124268
1,77242,"2900 tyler, Detroit MI",42.390729,-83.124268
2,77243,"2900 tyler, Detroit MI",42.390729,-83.124268
3,103945,"2900 tyler, Detroit MI",42.390729,-83.124268
4,138219,"2900 tyler, Detroit MI",42.390729,-83.124268


In [255]:
X = X.merge(addresses_and_latlons, on='ticket_id')
test_df = test_df.merge(addresses_and_latlons, on='ticket_id')

In [256]:
test_df.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,address,lat,lon
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,20.0,10.0,20.0,0.0,0.0,250.0,NaN,"10041 roseberry, Detroit MI",42.407581,-82.986642
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,20.0,10.0,100.0,0.0,0.0,1130.0,NaN,"18520 evergreen, Detroit MI",42.426239,-83.238259
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,20.0,10.0,10.0,0.0,0.0,140.0,NaN,"18520 evergreen, Detroit MI",42.426239,-83.238259
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,20.0,10.0,20.0,0.0,0.0,250.0,NaN,"1835 central, Detroit MI",42.309661,-83.122426
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,20.0,10.0,10.0,0.0,0.0,140.0,NaN,"1700 central, Detroit MI",42.308830,-83.121116


In [257]:
# remove columns to prevent data leakage
X.drop(columns=['payment_amount', 'payment_date', 'payment_status', 'balance_due', 'collection_status', 'compliance_detail'], inplace=True)

In [258]:
# drop redundant 'address' column after merge
X.drop(columns=['address'], inplace=True)
test_df.drop(columns=['address'], inplace=True)

In [259]:
print(X.shape)
X.head()

(250306, 29)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,250.0,20.0,10.0,25.0,0.0,0.0,305.0,NaN,42.390729,-83.124268
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,750.0,20.0,10.0,75.0,0.0,0.0,855.0,NaN,42.326937,-83.135118
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,42.380516,-83.096069
3,22084,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","MOROSI, MIKE",1441.0,LONGFELLOW,NaN,5.0,ST. CLAIR,DETROIT,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,42.380570,-83.095919
4,22093,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","NATHANIEL, NEAL",2449.0,CHURCHILL,NaN,7449.0,CHURCHILL,DETROIT,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,42.145257,-83.208233


In [260]:
# number of unique values
cols_to_encode = []
for col in X.columns:
    print(col, 'coltype - ', X[col].dtype, ' - ', len(X[col].unique()))
    if (X[col].dtype == 'object'):
        cols_to_encode.append(col)

ticket_id coltype -  int64  -  250306
agency_name coltype -  object  -  5
inspector_name coltype -  object  -  173
violator_name coltype -  object  -  119993
violation_street_number coltype -  float64  -  19175
violation_street_name coltype -  object  -  1791
violation_zip_code coltype -  float64  -  1
mailing_address_str_number coltype -  float64  -  15827
mailing_address_str_name coltype -  object  -  37897
city coltype -  object  -  5184
state coltype -  object  -  60
zip_code coltype -  object  -  5643
non_us_str_code coltype -  object  -  3
country coltype -  object  -  5
ticket_issued_date coltype -  object  -  86979
hearing_date coltype -  object  -  6223
violation_code coltype -  object  -  235
violation_description coltype -  object  -  258
disposition coltype -  object  -  9
fine_amount coltype -  float64  -  44
admin_fee coltype -  float64  -  2
state_fee coltype -  float64  -  2
late_fee coltype -  float64  -  37
discount_amount coltype -  float64  -  13
clean_up_cost colty

In [261]:
cols_to_encode

['agency_name',
 'inspector_name',
 'violator_name',
 'violation_street_name',
 'mailing_address_str_name',
 'city',
 'state',
 'zip_code',
 'non_us_str_code',
 'country',
 'ticket_issued_date',
 'hearing_date',
 'violation_code',
 'violation_description',
 'disposition',
 'grafitti_status']

In [262]:
X_encoded = X.copy()
test_df_encoded = test_df.copy()

for col in cols_to_encode:
    X_encoded[col] = X[col].astype('category').cat.codes
    test_df_encoded[col] = test_df[col].astype('category').cat.codes

In [263]:
X_encoded.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
0,22056,0,137,52230,2900.0,1640,NaN,3.0,27078,682,...,250.0,20.0,10.0,25.0,0.0,0.0,305.0,-1,42.390729,-83.124268
1,27586,0,166,77135,4311.0,306,NaN,2959.0,18442,1204,...,750.0,20.0,10.0,75.0,0.0,0.0,855.0,-1,42.326937,-83.135118
2,22062,0,137,94185,1449.0,1021,NaN,23658.0,21873,1104,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,42.380516,-83.096069
3,22084,0,137,74625,1441.0,1021,NaN,5.0,28889,1104,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,42.380570,-83.095919
4,22093,0,137,77944,2449.0,343,NaN,7449.0,5401,1104,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,42.145257,-83.208233


In [267]:
test_df_encoded.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
0,284932,1,45,12408,10041.0,1181,NaN,141,11750,739,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,-1,42.407581,-82.986642
1,285362,1,65,36291,18520.0,471,NaN,19136,5502,739,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,-1,42.426239,-83.238259
2,285361,1,65,36291,18520.0,471,NaN,19136,5502,739,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,-1,42.426239,-83.238259
3,285338,1,99,15025,1835.0,255,NaN,2246,9232,3040,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,-1,42.309661,-83.122426
4,285346,1,99,7990,1700.0,255,NaN,3435,8517,1558,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,-1,42.308830,-83.121116


In [264]:
print(X_encoded.isna().values.any())
print(X_encoded.isnull().values.any())
print(X_encoded.shape)

True
True
(250306, 29)


In [268]:
# missing values before handling
X_encoded.isnull().sum()

ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                      0
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           0
city                               0
state                              0
zip_code                           0
non_us_str_code                    0
country                            0
ticket_issued_date                 0
hearing_date                       0
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost                      0
judgment_amount                    0
grafitti_status                    0
l

In [244]:
test_df_encoded.isnull().sum()

ticket_id                         0
agency_name                       0
inspector_name                    0
violator_name                     0
violation_street_number           0
violation_street_name             0
violation_zip_code            36977
mailing_address_str_number     1014
mailing_address_str_name          0
city                              0
state                             0
zip_code                          0
non_us_str_code                   0
country                           0
ticket_issued_date                0
hearing_date                      0
violation_code                    0
violation_description             0
disposition                       0
fine_amount                       0
admin_fee                         0
state_fee                         0
late_fee                          0
discount_amount                   0
clean_up_cost                     0
judgment_amount                   0
grafitti_status                   0
address                     

In [269]:
X_encoded.mailing_address_str_number.fillna(value=0, inplace=True)
test_df_encoded.mailing_address_str_number.fillna(value=0, inplace=True)

X_encoded.drop(columns=['violation_zip_code'], inplace=True)
test_df_encoded.drop(columns=['violation_zip_code'], inplace=True)

X_encoded.fine_amount.fillna(value=0, inplace=True)

In [270]:
X_encoded.lat.fillna(value=0, inplace=True)
X_encoded.lon.fillna(value=0, inplace=True)
test_df_encoded.lat.fillna(value=0, inplace=True)
test_df_encoded.lon.fillna(value=0, inplace=True)

In [271]:
# missing values after handling
X_encoded.isnull().sum()

ticket_id                     0
agency_name                   0
inspector_name                0
violator_name                 0
violation_street_number       0
violation_street_name         0
mailing_address_str_number    0
mailing_address_str_name      0
city                          0
state                         0
zip_code                      0
non_us_str_code               0
country                       0
ticket_issued_date            0
hearing_date                  0
violation_code                0
violation_description         0
disposition                   0
fine_amount                   0
admin_fee                     0
state_fee                     0
late_fee                      0
discount_amount               0
clean_up_cost                 0
judgment_amount               0
grafitti_status               0
lat                           0
lon                           0
dtype: int64

In [272]:
test_df_encoded.isnull().sum()

ticket_id                     0
agency_name                   0
inspector_name                0
violator_name                 0
violation_street_number       0
violation_street_name         0
mailing_address_str_number    0
mailing_address_str_name      0
city                          0
state                         0
zip_code                      0
non_us_str_code               0
country                       0
ticket_issued_date            0
hearing_date                  0
violation_code                0
violation_description         0
disposition                   0
fine_amount                   0
admin_fee                     0
state_fee                     0
late_fee                      0
discount_amount               0
clean_up_cost                 0
judgment_amount               0
grafitti_status               0
lat                           0
lon                           0
dtype: int64

In [276]:
print(X_encoded.isna().values.any())
print(X_encoded.isnull().values.any())
print(X_encoded.shape)

False
False
(250306, 28)


In [277]:
print(X_encoded.shape)
X_encoded.head()

(250306, 28)


,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
0,22056,0,137,52230,2900.0,1640,3.0,27078,682,16,...,250.0,20.0,10.0,25.0,0.0,0.0,305.0,-1,42.390729,-83.124268
1,27586,0,166,77135,4311.0,306,2959.0,18442,1204,24,...,750.0,20.0,10.0,75.0,0.0,0.0,855.0,-1,42.326937,-83.135118
2,22062,0,137,94185,1449.0,1021,23658.0,21873,1104,24,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,42.380516,-83.096069
3,22084,0,137,74625,1441.0,1021,5.0,28889,1104,24,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,42.380570,-83.095919
4,22093,0,137,77944,2449.0,343,7449.0,5401,1104,24,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,42.145257,-83.208233


In [278]:
Y.fillna(value=2, inplace=True)

In [279]:
print(Y.shape)
Y.head()

(250306,)


0    0.0
1    1.0
2    2.0
3    2.0
4    2.0
Name: compliance, dtype: float64

In [280]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [281]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, random_state=0)

In [282]:
%%time
clf = RandomForestClassifier().fit(X_train, y_train)

C:\Users\Admin\Anaconda3\envs\TensorFlow-GPU-Keras\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Wall time: 3.32 s


In [283]:
y_predicted = clf.predict(X_test)

In [284]:
clf.score(X_test, y_test)

0.9645077264809755

In [285]:
print((y_predicted != y_test).sum())
print((y_predicted == y_test).sum())

2221
60356


In [286]:
((y_predicted != y_test).sum() / (y_predicted == y_test).sum()) * 100

3.6798329909205383

In [287]:
from sklearn.metrics import roc_auc_score

In [288]:
y_test_auc = y_test[y_test != 2.0]
y_predicted_auc = y_predicted[y_predicted != 2.0]

In [289]:
roc_auc_score(y_test_auc, y_predicted_auc)

0.6597769591105888

In [294]:
X_test.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
179489,210356,0,36,13109,10325.0,525,131.0,9480,1104,24,...,250.0,0.0,0.0,0.0,0.0,0.0,0.0,-1,42.445295,-83.165244
127587,154739,0,36,99400,700.0,1729,35122.0,33248,4601,24,...,300.0,20.0,10.0,30.0,0.0,0.0,360.0,-1,42.419677,-83.112109
8519,28080,0,38,90216,19194.0,319,2172.0,6273,309,24,...,250.0,20.0,10.0,25.0,0.0,0.0,305.0,-1,42.433636,-83.106899
106322,132479,0,52,12694,17455.0,613,15369.0,1549,1104,24,...,250.0,20.0,10.0,25.0,0.0,0.0,305.0,-1,42.421785,-83.079526
149941,179377,1,78,112657,20152.0,65,11584.0,34780,1104,24,...,3500.0,20.0,10.0,350.0,0.0,0.0,3880.0,-1,42.443402,-83.074685


In [293]:
test_df_encoded.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,mailing_address_str_number,mailing_address_str_name,city,state,...,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,lat,lon
0,284932,1,45,12408,10041.0,1181,141,11750,739,23,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,-1,42.407581,-82.986642
1,285362,1,65,36291,18520.0,471,19136,5502,739,23,...,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,-1,42.426239,-83.238259
2,285361,1,65,36291,18520.0,471,19136,5502,739,23,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,-1,42.426239,-83.238259
3,285338,1,99,15025,1835.0,255,2246,9232,3040,23,...,200.0,20.0,10.0,20.0,0.0,0.0,250.0,-1,42.309661,-83.122426
4,285346,1,99,7990,1700.0,255,3435,8517,1558,23,...,100.0,20.0,10.0,10.0,0.0,0.0,140.0,-1,42.308830,-83.121116


In [290]:
# predicting values for the test set
y_test_predicted = clf.predict(test_df_encoded)

ValueError: could not convert string to float: 'PO'